In [1]:
#r "nuget: Plotly.NET, 5.1.0"
#r "nuget: Plotly.NET.Interactive, 5.0.0"
#r "nuget: FSharp.Data, 6.6.0"

Installed Packages FSharp.Data, 6.6.0 Plotly.NET, 5.1.0 Plotly.NET.Interactive, 5.0.0

Loading extensions from `C:\Users\stefdantonio.EUROPE\.nuget\packages\plotly.net.interactive\5.0.0\lib\netstandard2.1\Plotly.NET.Interactive.dll`

In [2]:
open System.Drawing

let rgbToHsl (color : Color) =
    // Normalize R, G, B in the range [0..1]
    let r = float color.R / 255.0
    let g = float color.G / 255.0
    let b = float color.B / 255.0
    
    // Find min and max values among R, G, B
    let cMax = max g b |> max r
    let cMin = min g b |> min r
    let delta = cMax - cMin
    
    // Calculate lightness
    let l = (cMax + cMin) / 2.0
    
    // If all channels are the same, it's a shade of gray => S=0, H=0
    if delta = 0.0 then
        (0.0, 0.0, l)
    else
        // Calculate saturation
        let s =
            if l < 0.5 then
                delta / (cMax + cMin)
            else
                delta / (2.0 - cMax - cMin)
        
        // Calculate hue (base formula gives result in [0..6], we turn it into degrees [0..360])
        let hBase =
            match cMax with
            | _ when r = cMax -> (g - b) / delta % 6.0
            | _ when g = cMax -> (b - r) / delta + 2.0
            | _               -> (r - g) / delta + 4.0
        
        // Convert hBase to degrees: multiply by 60
        // ensure non-negative by adding 360 if negative
        let hDegrees =
            let raw = 60.0 * hBase
            if raw < 0.0 then raw + 360.0 else raw
        
        (hDegrees, s, l)

In [3]:
open FSharp.Data
open System.Drawing

let colorsCsvPath =
    System.IO.Directory.GetCurrentDirectory() + "/MaxMeyer.csv"

type MaxMeyerColor =
    { Name : string; Code : string; Color : Color }

let colorToHtml color =
    $"""<p>{color.Name}<br>{color.Code}<br>{color.Color.Name}</p>
        <svg width="300"
             height="200">
          <rect width="300" 
                height="200" 
                style="fill:rgb({color.Color.R},{color.Color.G},{color.Color.B})" />
        </svg>"""

let colorOptionToHtml =
    function
    | Some c -> c
    | None   -> { Name = "Not found"; Code = "N/A"; Color = Color.Black }
    >> colorToHtml

Formatter.Register<MaxMeyerColor>(colorToHtml, "text/html")
Formatter.Register<MaxMeyerColor option>(colorOptionToHtml, "text/html")
Formatter.Register<MaxMeyerColor option array>(
    (Array.map (colorOptionToHtml >> sprintf """<td>%s</td>""")
     >> Array.chunkBySize 5
     >> Array.map (String.concat "" >> sprintf "<tr>%s</tr>")
     >> String.concat ""
     >> fun rows -> $"<table>{rows}</table>"), 
    "text/html")

let colors =
    CsvFile.Load(colorsCsvPath).Rows
    |> Seq.map (fun row -> {
        Name  = if String.IsNullOrEmpty row.Columns[0] then row.Columns[1] else row.Columns[0]
        Code  = row.Columns[1]
        Color = Color.FromArgb(0, int row.Columns[2], int row.Columns[3], int row.Columns[4])})
    |> Array.ofSeq
    
let findColor (name : string) =
    let lowerName =
        name.ToLower()
    
    let tryFindMatchOn f () =
        Array.tryFind (fun color -> f color lowerName) colors

    colors
    |> Array.tryFind _.Name.ToLower().Contains(lowerName)
    |> Option.orElseWith (tryFindMatchOn _.Code.ToLower().Contains)
    |> Option.orElseWith (tryFindMatchOn _.Color.Name.ToLower().Contains)
    |> Option.orElseWith (tryFindMatchOn (fun color name -> $"{color.Color.R} {color.Color.G} {color.Color.B}" = name))

let findClosestColors mmColor =
    let r, g, b =
        mmColor.Color.R, mmColor.Color.G, mmColor.Color.B
    
    colors
    |> Array.sortBy (fun mmc ->
        let dR, dG, dB = 
            mmc.Color.R - r, mmc.Color.G - g, mmc.Color.B - b
        dR*dR + dG*dG + dB*dB)

let findClosestColorsHsl mmColor =
    let h, s, l =
        rgbToHsl mmColor.Color

    colors
    |> Array.sortBy (fun mmc ->
        let mh, ms, ml =
            rgbToHsl mmc.Color
        let dH = mh - h
        let dS = ms - s
        let dL = ml - l
        dH*dH + dS*dS + dL*dL)

let findClosestColorsHslFromName name =
    findColor name
    |> Option.map findClosestColorsHsl
    |> Option.defaultValue [||]

In [4]:
let colors = [|
    "cactus"
    "MAX19-0943"
|]

colors
|> Array.map findColor

MAX19-0942Cactus6d7a5c,MAX19-0943Aroma5a674d


In [5]:
findClosestColorsHslFromName "cactus"
|> Array.take 60
|> Array.map Some

MAX19-0942Cactus6d7a5c,MAX19-0928Verde Chiaro82b73c,MAX19-1093Zebucfd5c7,MAX19-0995Fumo Verdecfd5c7,MAX19-0922Dollaro506d29
MAX19-0919Verde Pop82af49,MAX19-0933Flanellabcd997,MAX19-0940Florab9cca1,CRF088-1CRF088-1b0c09c,MAX19-0930Verde Latifoglie5a8025
MAX19-0054Medicinad8ddd1,MAX19-0920granny smith75934a,MAX19-0912Finocchioaacf7e,VerdeVerde7ead3a,MAX19-0921La Sera dei Miracoli5a7639
MAX19-0931Friariello4e7a1a,MAX19-0913Ondulato9ec176,MAX19-0927Verde Selvaggio96c254,MAX19-0916Cerfoglio556b3c,MAX19-0926Tea treea2c967
MAX19-0938Primavera4b5b39,MAX19-0941Soprabosco849571,MAX19-0944Orgosolo444f38,MAX19-0935Aloe8da672,MAX19-0914Sospetto7f9f5d
MAX19-0929Verde Luminoso74a81e,MAX19-0943Aroma5a674d,MAX19-0936Verde Stagnante758c5e,MAX19-0937lacerta5f724c,CRF0117-1CRF0117-171736f
MAX19-0915Green Wheels68844c,MAX19-0917Agata3f522c,MAX19-0934Pigiamaa5c288,MAX19-0890Panna Cottac6d2ba,GrigioGrigioe9eae8
CRF004-1CRF004-1fcfefa,MAX19-0841Traslucidoe4e9db,MAX19-0904Valangae2e8d7,CRF084-2CRF084-29ba883,CRF083-3CRF083-3a3c06d
CRF087-1CRF087-1c3cbb4,MAX19-0924Pepe Verde3a5224,MAX19-0945Verde Pino313b28,CRF087-2CRF087-2a9b3a0,MAX19-0923Rapini435d2c
CRF084-3CRF084-3485234,RAL 7035Grigio Luceaeafac,MAX19-0862Equilibriod0d5c6,CRF006-2CRF006-2eef0ea,CRF005-1CRF005-1f4f6f0
MAX19-0055Salgemma Mineraled5d9cc,MAX19-0876Chitoned0e1c3,CRF003CRF003fbfef4,MAX19-0918Guayaberaccd4b9,Home Color - Armonia Sognatore 3Nebbiadde1da
